<a href="https://colab.research.google.com/github/Nandhagopalan/DLfromScratch/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import numpy as np
from sklearn import svm
from scipy.sparse import vstack
import pickle
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from pylab import rcParams
from keras.models import Model, load_model
from keras.layers import Input, Dense,Embedding,Flatten,LSTM,TimeDistributed
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping
from keras import regularizers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_curve,f1_score,precision_score
import numpy as np
from pandas import Series
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from keras import backend as K
import json
from keras.layers.normalization import BatchNormalization
import pickle

In [0]:
with open('train.pkl','rb') as f:
 data=pickle.load(f)

In [0]:
with open('test.pkl','rb') as f:
 test=pickle.load(f)

In [12]:
data.shape

(11216, 30)

In [32]:
nb_epoch = 100
batch_size = 128
timesteps = 30
embedding_length = 100
vocab=3198

inputs=Input(name='inputs',shape=[timesteps,])
embedding=Embedding(vocab, embedding_length, input_length=timesteps)(inputs)

encoded1=LSTM(64, activation='relu',return_sequences=True)(embedding)
encoded2=LSTM(32, activation='relu', return_sequences=True)(encoded1)
#encoded3=LSTM(128, activation='relu',return_sequences=True)(encoded2)


decoder1=LSTM(32, activation='relu',return_sequences=True)(encoded2)
decoder2=LSTM(64, activation='relu',return_sequences=True)(decoder1)
#decoder3=LSTM(512, activation='relu',return_sequences=True)(decoder2)
flat=Flatten()(decoder2)
output=Dense(timesteps)(flat)
#timedist=TimeDistributed(Dense(1))(flat)
#output=Flatten()(timedist)

sequence_autoencoder = Model(inputs, output)

sequence_autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

print(sequence_autoencoder.summary())

 #Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=20),
             ModelCheckpoint(filepath="ericsson_autoencoder.h5",
                               save_best_only=True,
                               verbose=0)]


history=sequence_autoencoder.fit(data, data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    validation_data=(test, test),
                    verbose=1,
                    callbacks=callbacks).history
                          
'''
Saving the json model
'''

with open('eric_def.json','w') as ff:
    json_string = sequence_autoencoder.to_json()
    ff.write(json_string)
                          

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 30)                0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 30, 100)           319800    
_________________________________________________________________
lstm_57 (LSTM)               (None, 30, 64)            42240     
_________________________________________________________________
lstm_58 (LSTM)               (None, 30, 32)            12416     
_________________________________________________________________
lstm_59 (LSTM)               (None, 30, 32)            8320      
_________________________________________________________________
lstm_60 (LSTM)               (None, 30, 64)            24832     
_________________________________________________________________
flatten_6 (Flatten)          (None, 1920)              0         
__________